<a href="https://colab.research.google.com/github/Jaime44/KUNVENO/blob/main/icefall_yes_no_dataset_recipe_with_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yesno recipe in icefall

This notebook shows you how to setup the environment to use [icefall][icefall] for training and decoding.
It also describes how to use a per-trained model to decode waves.


We use the [yesno] dataset as an example.

[icefall]: https://github.com/k2-fsa/icefall
[yesno]: https://www.openslr.org/1/

## Environment setup

### Install PyTorch and torchaudio

In [ ]:
import torch
print(torch.__version__)

Colab pre-installs PyTorch, so we don't need to install it here.

From https://pytorch.org/audio/main/installation.html#compatibility-matrix, we need to install torchaudio==2.0.2 as the current PyTorch version is 2.0.1

In [ ]:
# ! pip install torchaudio==2.0.2

### Install k2

We are going to install k2 by following https://k2-fsa.github.io/k2/installation/from_wheels.html.


In [ ]:
# ! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Check that k2 was installed successfully:

In [ ]:
! python3 -m k2.version

### Install lhotse
[lhotse][lhotse] is used for data preparation.

[lhotse]: https://github.com/lhotse-speech/lhotse

Normally, we would use `pip install lhotse`. However, the yesno recipe is added recently and has not been released to PyPI yet, so we install the latest unreleased version here.

In [ ]:
# ! pip install lhotse
# ! pip install git+https://github.com/lhotse-speech/lhotse

### Install icefall

[icefall][icefall] is a collection of Python scripts.
You don't need to install it. What you need to do is
to get its source code, install its dependencies, and
set the `PYTHONPATH` pointing to it.

[icefall]: https://github.com/k2-fsa/icefall

In [ ]:
! pwd

In [ ]:
# ! git clone https://github.com/k2-fsa/icefall

Now install dependencies of `icefall`:

In [ ]:
! cd icefall && \
  pip install -r requirements.txt

## Data preparation

We have set up the environment. Now it is the time to prepare data for training and decoding.

As we just said, `icefall` is a collection of Python scripts and we have to set up the `PYTHONPATH` variable to use it. Remember that `icefall` was downloaded to
`/content/icefall`, so we use

```
export PYTHONPATH=/content/icefall:$PYTHONPATH
```

**HINT**: You can have several versions of `icefall` in your virtual environemnt. To switch to a specific version of `icefall`, just change the `PYTHONPATH` environment variable.

In [ ]:
# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

In [ ]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  rm -rf data && \
  ./prepare.sh

## Training

In [ ]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/train.py

## Decoding

In [ ]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/decode.py

### Show the decoding result

In [ ]:
! cd /content/icefall/egs/yesno/ASR && \
  cat tdnn/exp/recogs-test_set.txt

### Show the detailed WER

In [ ]:
! cd /content/icefall/egs/yesno/ASR && \
  cat tdnn/exp/errs-test_set.txt

# Pre-trained model

### Download the pre-trained model

In [ ]:
! cd /content/icefall/egs/yesno/ASR && \
  mkdir tmp && \
  cd tmp && \
  git lfs install && \
  git clone https://huggingface.co/csukuangfj/icefall_asr_yesno_tdnn

In [ ]:
! sudo apt-get install git-lfs

In [ ]:
! cd /content/icefall/egs/yesno/ASR && \
  mkdir -p tmp && \
  tree tmp

In [ ]:
! sudo apt-get install tree

In [ ]:
! cd /content/icefall/egs/yesno/ASR && \
  mkdir -p tmp && \
  tree tmp

In [ ]:
! cd /content/icefall/egs/yesno/ASR && \
  soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

In [ ]:
! sudo apt-get install sox

In [ ]:
! cd /content/icefall/egs/yesno/ASR && \
  soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

## Download kaldifeat

See https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html

In [ ]:
! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

## Inference with a pre-trained model


### View help information

In [ ]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py --help

### Decode a single sound file

In [ ]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
    --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
    --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

### Decode multiple sound files

In [ ]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
    --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
    --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav